# Transform Odysea data to DGGS

## Context

### Purpose

ODYSEA (Ocean DYnamics and Surface Exchange with the Atmosphere) is a proposed French-U.S. satellite 
mission designed to study wind-ocean surface interactions. Developed by the Jet Propulsion Laboratory (JPL) and CNES, ODYSEA aims to provide critical insights into the dynamics of ocean-atmosphere exchanges, enhancing our understanding of Earth's climate system. The mission has been proposed for NASA’s Earth System Explorers Announcement of Opportunity (AO), highlighting its potential to advance oceanographic and atmospheric research.

While these types of remote sensing data offer excellent coverage, there are still gaps in the data that need to be addressed. Additionally, the data is complex due to overlaps, and each scan corresponds to a slightly different time based on the satellite's track.

### Description

In this notebook, we will:
- Access, open and prepare the Odysea sample dataset as a xarray datatree
- Transform the Odysea dataset into DGGS Healpix
- Save thetransformed dataset in Zarr
- Visualize the transformed dataset with `xdggs`

## Contributions

### Notebook

- Jean-Marc Delouis, IFREMER (France), @jmdelouis
- Anne Fouilloux, Simula Research Laboratory (Norway) (reviewer)

## Bibliography and other interesting resources

- [ODYSEA](https://cnes.fr/en/projects/odysea)
- [xdggs](https://xdggs.readthedocs.io/en/latest/)

```{warning}
This notebook has been developed and tested on Datamor (HPC from IFREMER).
```

In [1]:
#!pip install xdggs pint_xarray icechunk
!pip install xdggs
!pip install healpy
!pip install -U zarr==2.18.4

In [1]:
import warnings
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xdggs
import fsspec
import sparse

warnings.filterwarnings("ignore")
xr.set_options(display_max_rows=27)

In [12]:
from numcodecs import Zstd


class proj_odysea:

    def __init__(
        self,
        nside,
        nscale=2,
        sz_orbit=100000,
        nest=False,
        chunk_size=4096,
        cell_id_name="cell_ids",
    ):
        self.nside = nside
            
        self.nscale = nscale
        self.sz_orbit = sz_orbit
        self.nest = nest
        self.chunk_size = chunk_size
        self.cell_id_name = cell_id_name
        # Configuration du compresseur (Zstandard) avec la valeur de remplissage
        self.mask={}
        gebco_path='GEBCO_2048.fits'
        gebco_path='/home/lops-oh-fair2adapt/gebco/GEBCO_2048.fits'
        
        for k in range(self.nscale+1):
            if nside//(2**(k))<2048:
                self.mask[k] = np.full(12 * (nside//(2**(k)))**2, np.nan)
                self.mask[k][hp.reorder(hp.ud_grade(hp.read_map(gebco_path),(nside//(2**(k)))),r2n=True)<0]=1.0
            
    def setalldata(self, val, cell_ids,nside):
        nscale=self.nscale
        ldata = np.full(12 * nside**2, np.nan)
        ldata[cell_ids] = val
        if nscale==0:
            return (self.mask[0]*ldata).reshape(12 * nside**2, 1)
            
        hdata=~np.isnan(ldata)
        val={}
        sdata={}
        for k in range(nscale):
            val[k]=np.nanmean((ldata).reshape(12*(nside//2**(k+1))**2,4**(k+1)),1)
            sdata[k]=np.full(12 * (nside//2**(k+1))**2, np.nan)
            
        data=hp.reorder(ldata,n2r=True)
        n=0
        for k in range(nside-1):
            N=4*(k+1)
            idx=np.where(~np.isnan(data[n:n+N]))[0]
            if len(idx)>0:
                beg=np.min(idx)+n
                ed=np.max(idx)+n
                idx2=np.where(np.isnan(data[beg:ed]))[0]
                l=0
                while len(idx2)>0 and l<nscale:
                    idx2r=hp.ring2nest(nside,idx2+beg)
                    data[idx2+beg]=val[l][(idx2r)//(4**(l+1))]
                    sdata[l][(idx2r)//(4**(l+1))]=val[l][(idx2r)//(4**(l+1))]
                    idx2=np.where(np.isnan(data[beg:ed]))[0]
                    l=l+1
            n=n+N
    
        for k in range(2*nside+1):
            N=4*nside
            idx=np.where(~np.isnan(data[n:n+N]))[0]
            if len(idx)>0:
                beg=np.min(idx)+n
                ed=np.max(idx)+n
                idx2=np.where(np.isnan(data[beg:ed]))[0]
                l=0
                while len(idx2)>0 and l<nscale:
                    idx2r=hp.ring2nest(nside,idx2+beg)
                    data[idx2+beg]=val[l][(idx2r)//(4**(l+1))]
                    sdata[l][(idx2r)//(4**(l+1))]=val[l][(idx2r)//(4**(l+1))]
                    idx2=np.where(np.isnan(data[beg:ed]))[0]
                    l=l+1
            n=n+N
    
        for k in range(nside-1):
            N=4*(nside-1-k)
            idx=np.where(~np.isnan(data[n:n+N]))[0]
            if len(idx)>0:
                beg=np.min(idx)+n
                ed=np.max(idx)+n
                idx2=np.where(np.isnan(data[beg:ed]))[0]
                l=0
                while len(idx2)>0 and l<nscale:
                    idx2r=hp.ring2nest(nside,idx2+beg)
                    data[idx2+beg]=val[l][(idx2r)//(4**(l+1))]
                    sdata[l][(idx2r)//(4**(l+1))]=val[l][(idx2r)//(4**(l+1))]
                    idx2=np.where(np.isnan(data[beg:ed]))[0]
                    l=l+1
            n=n+N
        for k in range(nscale):
            sdata[k]=(self.mask[k+1]*sdata[k]).reshape(12 * (nside//(2**(k+1)))**2, 1)
        return (self.mask[0]*ldata).reshape(12 * nside**2, 1),sdata

    def eval(self, url, datarmor, angle=[], time=[], orbit=0):
        
        if(datarmor):
            ds = xr.open_dataset(url, engine="h5netcdf")
        else:
            fs = fsspec.filesystem("http")
            ds = xr.open_dataset(fs.open(url), engine="h5netcdf")

        
        print("Orbit ", url)

        var_name = list(ds.data_vars)

        mask = ~np.isnan(ds.u_model.data)

        ds_total={}

        nside=self.nside
        
        hidx = hp.ang2pix(
            nside,
            ds.lon.values[mask],
            ds.lat.values[mask],
            lonlat=True,
            nest=self.nest,
        ).flatten()
    
        him = np.bincount(hidx, minlength=12 * nside**2)
    
        cell_ids = np.where(him > 0)[0]

        out_map={}
        for k in range(self.nscale+1):
            out_map[k]= []
            
        healpix_ids = np.arange(12 * nside**2)
        var_cell_ids = xr.DataArray(
                healpix_ids,
                dims="cells",
                attrs={"grid_name": "healpix", "nside": nside, "nest": self.nest},
        )
        svar_cell_ids={}
        for k in range(self.nscale):
            svar_cell_ids[k] = xr.DataArray(
                np.arange(12 * (nside//(2**(k+1)))**2),
                dims="cells",
                attrs={"grid_name": "healpix", "nside": nside//(2**(k+1)), "nest": self.nest},
            )
    
        for ivar in var_name:
            if len(ds[ivar].values.shape) == 2:
                if ivar in angle:
                    c_hmap = np.bincount(
                        hidx,
                        weights=np.cos(ds[ivar].values[mask].flatten()),
                        minlength=12 * nside**2,
                    )
                    s_hmap = np.bincount(
                        hidx,
                        weights=np.sin(ds[ivar].values[mask].flatten()),
                        minlength=12 * nside**2,
                    )
                    data,sdata = self.setalldata(
                        np.arctan2(s_hmap[cell_ids], c_hmap[cell_ids]), cell_ids,nside
                    )
                elif ivar in time:
                    print("donothing", ivar)
                    data,sdata = self.setalldata(np.zeros([cell_ids.shape[0]]), cell_ids,nside)
                else:
                    data = np.bincount(
                        hidx,
                        weights=ds[ivar].values[mask].flatten(),
                        minlength=12 * nside**2,
                    )
                    data,sdata = self.setalldata(data[cell_ids] / him[cell_ids], cell_ids,nside)
    
                out_map[0].append(
                    xr.DataArray(
                        data,
                        dims=("cells", "orbit"),
                        coords={self.cell_id_name: var_cell_ids, "orbit": [orbit]},
                        name=ivar,
                    )
                )
                for k in range(self.nscale):
                    out_map[k+1].append(
                        xr.DataArray(
                            sdata[k],
                            dims=("cells", "orbit"),
                            coords={self.cell_id_name: svar_cell_ids[k], "orbit": [orbit]},
                            name=ivar,
                        )
                    )
    
        ivar = "hit"
        data,sdata =self.setalldata(him[cell_ids], cell_ids,nside)
        out_map[0].append(
            xr.DataArray(
                data,
                dims=("cells", "orbit"),
                coords={self.cell_id_name: var_cell_ids, 
                        "orbit": [orbit]},
                name=ivar,
            )
        )
        for k in range(self.nscale):
            out_map[k+1].append(
                xr.DataArray(
                    sdata[k],
                    dims=("cells", "orbit"),
                    coords={self.cell_id_name: svar_cell_ids[k], "orbit": [orbit]},
                    name=ivar,
                )
            )
        print("Npt[%d] = "%(nside), len(cell_ids))

        for k in range(self.nscale+1):
            ds_total[k] = xr.merge(out_map[k])
            chunk_size_data = int(( 12 * (4**level)) / self.chunk_size)

#            chunk_size_data = int(self.chunk_size)
            print("chunk_size_data", chunk_size_data)
            ds_total[k] = ds_total[k].chunk({"cells": chunk_size_data})

        return ds_total

    def plot(self, u, v):
        plt.figure(figsize=(12, 4))
        hp.cartview(
            u,
            nest=True,
            hold=False,
            sub=(1, 2, 1),
            title="U Model",
            min=-1,
            max=1,
            unit=r"$m.s^{-1}$",
            cmap="jet",
            flip="geo",
        )
        hp.cartview(
            v,
            nest=True,
            hold=False,
            sub=(1, 2, 2),
            title="V Model",
            min=-1,
            max=1,
            unit=r"$m.s^{-1}$",
            cmap="jet",
            flip="geo",
        )

In [32]:
from numcodecs import Zstd

level = 10
nside=2**level
nside_list=2**level
nest=True
chunk_size = 12 *(4**0)#3#*(2**level)
chunk_size_data = int(( 12 * (4**level)) / chunk_size)
print("chunk_size", chunk_size)
        
zstd_compressor = Zstd(level=3)

# Define a common compression setting
compression_settings = {"compressor": zstd_compressor,}# "chunks": (chunk_size, 1)}
min_orbit=161
max_orbit=274
max_orbit=163
max_orbit=200


chunk_size 12


In [ ]:

pr=proj_odysea(nside,
               nest=nest,
               chunk_size= chunk_size,
              nscale=2)
for k in range(min_orbit,max_orbit):
    url_gebco="https://data-fair2adapt.ifremer.fr/odysea/odysea/L2_global_mitgcm_confF_c13/odysea_L2_global_mitgcm_confF_c13_p%d.nc"%(k)
    url_gebco="/home/lops-oh-fair2adapt/odysea/odysea/L2_global_mitgcm_confF_c13/odysea_L2_global_mitgcm_confF_c13_p%d.nc"%(k)
    ds_all_data=pr.eval(url_gebco,datarmor=True,orbit=k,
                    angle=['lat','lon','wind_dir'],
                     time=['sample_time'])
    for l in range(len(ds_all_data)):
        ds_data = ds_all_data[l].fillna(hp.UNSEEN)

        l_level=level-l
        
        # Create the encoding dictionary for all variables
        encoding = {var: compression_settings for var in ds_data.data_vars}
    
        if nest:
            name='nest'
        else:
            name='ring'
        url_zarr="/home/jovyan/Akathon-Geilo/odysea_%d_%s.zarr"%(l_level,name)   
        url_zarr="/home/lops-oh-fair2adapt/odysea/L2_global_mitgcm_conf_healpix/F_c13_%d_%s.zarr"%(l_level,name)   
        url_zarr="/home/lops-oh-fair2adapt/odysea/test/F_c13_%d_%s.zarr"%(l_level,name)   
        
        if k==161:
            ds_data.to_zarr(url_zarr, 
                            mode="w",  #consolidated=True,
                            encoding=encoding)
        else:
            ds_data.to_zarr(url_zarr, 
                            mode="a", 
                            append_dim="orbit") #,consolidated=True)

Orbit  /home/lops-oh-fair2adapt/odysea/odysea/L2_global_mitgcm_confF_c13/odysea_L2_global_mitgcm_confF_c13_p161.nc
donothing sample_time


In [ ]:
#plevels = {"/": root, } "9": ds, "8": ds_8, "7": ds_7}
plevels = {}

for l in range(len(ds_all_data)):
    ds_data = ds_all_data[l].fillna(hp.UNSEEN)

    l_level=level-l

    # Create the encoding dictionary for all variables
    encoding = {var: compression_settings for var in ds_data.data_vars}

    if nest:
        name='nest'
    else:
        name='ring'
    url_zarr="/home/jovyan/Akathon-Geilo/odysea_%d_%s.zarr"%(l_level,name)   
    url_zarr="/home/lops-oh-fair2adapt/odysea/L2_global_mitgcm_conf_healpix/F_c13_%d_%s.zarr"%(l_level,name)   
#    url_zarr="/home/lops-oh-fair2adapt/odysea/test/F_c13_%d_%s.zarr"%(l_level,name)   
    print('loading',l_level,url_zarr)
    plevels[str(l_level)] = xr.open_zarr(url_zarr, 
                            #mode="w",  #consolidated=True,
                          #  encoding=encoding
               )
ds_all = xr.DataTree.from_dict(plevels) 
url_zarr="/home/jovyan/Akathon-Geilo/odysea_%s_%s.zarr"%('all',name)  
url_zarr="/home/lops-oh-fair2adapt/odysea/L2_global_mitgcm_conf_healpix/F_c13_%s_%s.zarr"%('all',name)   
#url_zarr="/home/lops-oh-fair2adapt/odysea/test/F_c13_%s_%s.zarr"%('all',name) 
#
print(url_zarr)
ds_all.to_zarr(url_zarr, mode="w",)


In [31]:
#dtotal=xr.open_zarr("odyssea.zarr").rename_vars(dict(cells="cell_ids")).assign_coords(cell_ids=lambda ds : ds["cell_ids"].assign_attrs({"grid_name":"healpix","nside":256,"nest":True})).pipe(xdggs.decode)
dtotal=xr.open_datatree(url_zarr)
dtotal

<xarray.DataTree>
Group: /
├── Group: /10
│       Dimensions:               (cells: 12582912, orbit: 2)
│       Coordinates:
│           cell_ids              (cells) int64 101MB ...
│         * orbit                 (orbit) int64 16B 161 162
│       Dimensions without coordinates: cells
│       Data variables:
│           azimuth_aft           (cells, orbit) float64 201MB ...
│           azimuth_fore          (cells, orbit) float64 201MB ...
│           encoder_aft           (cells, orbit) float64 201MB ...
│           encoder_fore          (cells, orbit) float64 201MB ...
│           hit                   (cells, orbit) float64 201MB ...
│           lat                   (cells, orbit) float64 201MB ...
│           lon                   (cells, orbit) float64 201MB ...
│           radial_angle_aft      (cells, orbit) float64 201MB ...
│           radial_angle_fore     (cells, orbit) float64 201MB ...
│           sample_time           (cells, orbit) float64 201MB ...
│           u_model               (cells, orbit) float64 201MB ...
│           ur_ac                 (cells, orbit) float64 201MB ...
│           ur_aft                (cells, orbit) float64 201MB ...
│           ur_al                 (cells, orbit) float64 201MB ...
│           ur_eastward           (cells, orbit) float64 201MB ...
│           ur_fore               (cells, orbit) float64 201MB ...
│           ur_nonoise_ac         (cells, orbit) float64 201MB ...
│           ur_nonoise_aft        (cells, orbit) float64 201MB ...
│           ur_nonoise_al         (cells, orbit) float64 201MB ...
│           ur_nonoise_eastward   (cells, orbit) float64 201MB ...
│           ur_nonoise_fore       (cells, orbit) float64 201MB ...
│           ur_nonoise_northward  (cells, orbit) float64 201MB ...
│           ur_northward          (cells, orbit) float64 201MB ...
│           v_model               (cells, orbit) float64 201MB ...
│           wind_dir              (cells, orbit) float64 201MB ...
│           wind_speed            (cells, orbit) float64 201MB ...
├── Group: /8
│       Dimensions:               (cells: 786432, orbit: 2)
│       Coordinates:
│           cell_ids              (cells) int64 6MB ...
│         * orbit                 (orbit) int64 16B 161 162
│       Dimensions without coordinates: cells
│       Data variables:
│           azimuth_aft           (cells, orbit) float64 13MB ...
│           azimuth_fore          (cells, orbit) float64 13MB ...
│           encoder_aft           (cells, orbit) float64 13MB ...
│           encoder_fore          (cells, orbit) float64 13MB ...
│           hit                   (cells, orbit) float64 13MB ...
│           lat                   (cells, orbit) float64 13MB ...
│           lon                   (cells, orbit) float64 13MB ...
│           radial_angle_aft      (cells, orbit) float64 13MB ...
│           radial_angle_fore     (cells, orbit) float64 13MB ...
│           sample_time           (cells, orbit) float64 13MB ...
│           u_model               (cells, orbit) float64 13MB ...
│           ur_ac                 (cells, orbit) float64 13MB ...
│           ur_aft                (cells, orbit) float64 13MB ...
│           ur_al                 (cells, orbit) float64 13MB ...
│           ur_eastward           (cells, orbit) float64 13MB ...
│           ur_fore               (cells, orbit) float64 13MB ...
│           ur_nonoise_ac         (cells, orbit) float64 13MB ...
│           ur_nonoise_aft        (cells, orbit) float64 13MB ...
│           ur_nonoise_al         (cells, orbit) float64 13MB ...
│           ur_nonoise_eastward   (cells, orbit) float64 13MB ...
│           ur_nonoise_fore       (cells, orbit) float64 13MB ...
│           ur_nonoise_northward  (cells, orbit) float64 13MB ...
│           ur_northward          (cells, orbit) float64 13MB ...
│           v_model               (cells, orbit) float64 13MB ...
│           wind_dir              (cells, orbit) float64 13MB ...
│           wi